In [1]:
import pickle
import numpy
import torch
from tqdm import tqdm

In [3]:
with open("../filtered_datasets_6-14-18/train_99.pkl", 'rb') as f:
    train_data = pickle.load(f)
with open("../filtered_datasets_6-14-18/validate_99.pkl", 'rb') as f:
    valid_data = pickle.load(f)

In [4]:
train_sps = list(set([i[0] for i in train_data]))
valid_sps = list(set([i[0] for i in valid_data]))

In [5]:
train_sps[:30]

['MKFAAILLVTCVLFSLLPSHLSQG',
 'MRITLPLLSTAVGLGLTAAVLGTGPAATA',
 'MFLKGYTSNVVLIILTFFILLTKE',
 'MHHDNFRRLGNAAFAAAAALLAVA',
 'MIQVLLVTVCLAVFPYQGSS',
 'MKLLSVCAIALLAAQAAG',
 'MILLHFVYSLWALLLIPLINA',
 'MTSFCSMISLLLLLSLSSAVFS',
 'MAMISYSTIVVALLASFMICSVSA',
 'MAPSQKALLVLVLSMLLTASDSWA',
 'MGTGAADGSRGARRWLPWLGLFFWAAGAAA',
 'MKFTLTWRCFALFLLLHQPTPV',
 'MWGLKVLLLPVMSFA',
 'MSGYEKPSRGWGFCALSPVLLSLLMA',
 'MKISFLLLALVICSIGWSEA',
 'MPRQSGFGWAWRVPLALAGSLAAAT',
 'MPAGTAARAWVLVLALWGAVAG',
 'MRLPLPLLLLFGCRAILGSFG',
 'MKGTAASALLIALSATAAQA',
 'MFTVFLLVVLATTVGS',
 'MKLFKKLGILLLITSLILLAA',
 'MKPWAAFHLIFLVASSLEG',
 'MRNLSTWPTFAALLWSAPRVLA',
 'MYQSTLKTILLASALLILPASMSA',
 'MNRLSCFLLVIGLCIGLSNA',
 'MSFIRSALAAAAFVALSIGAVQTASA',
 'MGNFTVRATAGLMLASLSTLAHA',
 'MKNKTNLGLSVFFFFICLVSVTS',
 'MKASRFLALAGLVLLFVVGYA',
 'MLCGPLCRFLWLWPYLSYVEA']

In [6]:
try:
    char_dict = pickle.load(open('vae_sp_char_dict.p','rb'))
except:
    raise Error
#     chars = set()
#     for sp in train_sps:
#         sp_chars = [char for char in sp]
#         chars.update(sp_chars)
#     for sp in valid_sps:
#         sp_chars = [char for char in sp]
#         chars.update(sp_chars)
#     chars.add('<START>')
#     chars.add('<END>')
#     chars.add('<PAD>')
#     chars = list(chars)
#     chars.sort()
#     char_dict = {char:idx for idx, char in enumerate(chars)}
#     pickle.dump(char_dict, open('vae_sp_char_dict.p','wb'))

In [7]:
def pad(seq, length=70):
    seq = [s for s in seq]
    seq = ['<START>'] + seq + ['<END>']
    
    while len(seq) < length:
        seq = seq + ['<PAD>']
    
    if len(seq)>length:
        return None
    else:
        return seq

def one_hot(seq, char_dict=char_dict, length=70):
    seq = pad(seq, length=length)
    if seq==None:
        return None
    else:
        embedding = torch.zeros(len(seq), len(char_dict))
        for idx, char in enumerate(seq):
            embedding[idx,char_dict[char]] = 1
        return embedding

length = 50
training_data = []
for sp in tqdm(train_sps):
    if len(sp) <= length-2:
        one_hot_sp = one_hot(sp, length=length)
        training_data.append(one_hot_sp)

valid_data = []
for sp in tqdm(valid_sps):
    if len(sp) <= length-2:
        one_hot_sp = one_hot(sp, length=length)
        valid_data.append(one_hot_sp)

100%|██████████| 5626/5626 [00:03<00:00, 1755.20it/s]


In [8]:
len(training_data)

21939

In [9]:
from torch.utils.data import DataLoader, Dataset

In [10]:
training_data = torch.stack(training_data)
valid_data = torch.stack(valid_data)

In [11]:
torch.save(training_data, 'train99_tensors.pt')
torch.save(valid_data, 'valid99_tensors.pt')

In [12]:
train_loader = DataLoader(training_data, shuffle=True, batch_size=32)

In [123]:
for i_batch, sample_batched in enumerate(train_loader):
    a,b = i_batch, sample_batched

In [126]:
b.shape

torch.Size([13, 50, 24])

In [135]:
50*24

1200

In [136]:
153600/1200

128.0

In [146]:
a= torch.tensor([[1,2,3,2],[4,3,5,3]])
a.argmax()

tensor(6)

In [153]:
a = torch.randn((2,3,5))

In [151]:
a.shape
a = a.view(2,2,2)
a.shape

torch.Size([2, 2, 2])

In [157]:
for i,seq in enumerate(a):
    for j,pos in enumerate(seq):
        print(a[i,j,:])
        print(pos.argmax())
        z = pos.argmax()
        print('-')

tensor([-0.1574, -0.4509, -0.4063,  1.1420,  0.6461])
tensor(3)
-
tensor([-0.1304, -0.2496, -0.7058, -0.1843,  0.6006])
tensor(4)
-
tensor([-0.0611,  0.1677, -0.4580, -0.5290, -0.6370])
tensor(1)
-
tensor([ 0.8457,  0.1528,  0.5613, -0.5172,  0.6991])
tensor(0)
-
tensor([-1.1948, -0.0167,  0.1681,  1.2170, -1.8672])
tensor(3)
-
tensor([-0.1641,  1.4369, -1.0226, -1.0374, -0.2292])
tensor(1)
-


In [161]:
z.item()

1

In [163]:
a = 'bdc'
a = a.replace('b','z')
a

'zdc'